# Here are the learnings from LangChain

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

[SWE-Best Practise]

dotenv is used when we have a `.env` file in the source repository. It is used to hide secrets, passwords and API tokens instead of coding it directly into python scripts, which you would naturally hard-code in. 

# Module 1: Agent Foundations

Here are a list of foundation tools with Langchain.

Requirements:
- `.env` file with API tokens to LLM model and Langchain.

In [ ]:
from langchain.chat_models import init_chat_model

# init_chat_model initiates the LLM model you will use
model = init_chat_model(model="gpt-5-nano")

# invoke triggers the LLM with your message prompt
response = model.invoke("What's inside the day?")
response

In [ ]:
# outputs the LLM's response
print(response.content)

from pprint import pprint

# pretify output for the metadata including model name and token usage
pprint(response.response_metadata)

In [ ]:
# models can be customised with the following parameters
model = init_chat_model(
    model="gpt-5-nano",
    # Kwargs passed to the model:
    temperature=1.0
)

The following parameters allows customisable LLM responses:
- temperature = control's the model's creativity
    - 0 - 0.2 = deterministic - used for math
    - 0.3 - 0.6 = focused - used for general QA
    - 0.7 - 1.0 = creative - used for creative writing

- max_tokens = how long the model's response will be
    - 0 - 50 = short answers
    - 51 - 500 = short explanations
    - 1000+ = long explanations

- timeout = how long the client waits before aborting request
    - 0 - 30 = Fast API calls
    - 30 - 60 = Long outputs

- max_retries = how many times to re attempt before cancelling the request
    - 0 - 2 = minimal
    - 3 - 5 = production safe

In [ ]:
init_chat_model(model="claude-sonnet-4-5")
ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

LangChain benefits from being model agnostic, they host the models so we can experiment with different ones all the time.

# Agent Foundations

In [ ]:
from langchain.agents import create_agent

# creates an agent focused on chat behaviour
agent = create_agent(model="gpt-5-nano")

In [ ]:
from langchain.messages import AIMessage, HumanMessage

response = agent.invoke(
    # Human prompt is first here
    {"messages": [HumanMessage(content="What's the capital of the Moon?"),
    AIMessage(content="The capital of the Moon is Luna City."),
    HumanMessage(content="Interesting, tell me more about Luna City")]}
)

pprint(response)

"messages" is a dictionary where Human and AI messages are stored for contexual learning and understanding for the AI. It can understand what's been said before in order to not repeat itself.

In [ ]:
for token, metadata in agent.stream(
    {"messages": [HumanMessage(content="Tell me all about Luna City, the capital of the Moon")]},
    stream_mode="messages"
):

    # token is a message chunk with token content
    # metadata contains which node produced the token
    
    if token.content:  # Check if there's actual content
        print(token.content, end="", flush=True)  # Print token

"stream" is used to stream the AI message back to the human so the percieved time is shorter than it actually is. AI message response is measured in number of seconds due to the time it takes rather than miliseconds in normal API calls.

## Agent Configurations

In [ ]:
system_prompt = "You are a science fiction writer, create a capital city at the users request."

# agent with the configurations
scifi_agent = create_agent(
    model="gpt-5-nano",
    system_prompt=system_prompt
)

# question invoked by human
question = HumanMessage(content="What's the capital of the moon?")
response = scifi_agent.invoke( # scifi agent with the system prompt
    {"messages": [question]}
)

print(response['messages'][1].content)

`system_prompt` allows the model to be configured to how you want the model to behave. This is a form of prompt engineering.

In [ ]:
# basic prompting
system_prompt = "You are a science fiction writer, create a capital city at the users request."

# example based prompting
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

User: What is the capital of mars?
Scifi Writer: Marsialis

User: What is the capital of Venus?
Scifi Writer: Venusovia

"""

# structured prompting
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

Please keep to the below structure.

Name: The name of the capital city

Location: Where it is based

Vibe: 2-3 words to describe its vibe

Economy: Main industries

"""


In [ ]:
# structured prompting with OOP classes
from pydantic import BaseModel

# remember a class is like a Microsoft Forms, fixed categories that all users must follow
class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

agent = create_agent(
    model='gpt-5-nano',
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
    response_format=CapitalInfo
)
question = HumanMessage(content="What is the capital of The Moon?")

response = agent.invoke(
    {"messages": [question]}
)

response["structured_response"]

In [ ]:
capital_info = response["structured_response"]

capital_name = capital_info.name
capital_location = capital_info.location

print(f"{capital_name} is a city located at {capital_location}")

Depending on the use case you can use OOP classes to allow the LLM to associate with the class attributes like name or location. 

# Agent Tools
Tools allows agents to access data, execute tasks and call other agents. 

In [5]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

In [6]:
# the name and description can be override if you prefer
@tool("square_root", description="Calculate the square root of a number")
def tool1(x: float) -> float:
    return x ** 0.5

In [ ]:
# functions can be invoked with the parameter and the value
tool1.invoke({"x": 467})

`@tool` decorator turns the function into a tool the agent can use. The function name and docstring needs to be detailed for the LLM to understand what the function does.

LLM's will call these functions with the invoke method. Invoke takes the parameters and and value for that parameter

Remember, a decorator sits above a function and can wrap a function into certain behaviours or classes.

In [7]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pprint import pprint


agent = create_agent(
    model="gpt-5-nano",
    tools=[tool1],
    system_prompt="You are an arithmetic wizard. Use your tools to calculate the square root and square of any number."
)

question = HumanMessage(content="What is the square root of 467?")

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

pprint(response['messages'])

# tool_calls used to look at the tool message
print(response["messages"][1].tool_calls)

√467 ≈ 21.61018278497431

Rounded: ≈ 21.6102 (4 decimal places).
[HumanMessage(content='What is the square root of 467?', additional_kwargs={}, response_metadata={}, id='c0dfb252-944f-4dc0-8310-ec10d9b69426'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1240, 'prompt_tokens': 158, 'total_tokens': 1398, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1216, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpiFaLmDhIhnpPJYxa7Ztxjn5U295', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4814-837e-72a2-91f7-c0e3fc38a19d-0', tool_calls=[{'name': 'square_root', 'args': {'x': 467}, 'id': 'call_msX6uPgYhCn9O4LmuAPfCZx6', 'type': 'tool_call'}], usage_metadata={'input_t

Agents are typically trained pre-dated so they will not have the most up to date information. You can simply ask the model:
`question = HumanMessage(content="How up to date is your training knowledge?")`

### Add web search tool

In order to get around this, we can set the agent with web tool so it can search for relevant information from the web.

In [8]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

# tavily search API is used to search the web in a LLM friendly way
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

web_search.invoke("Who is the current mayor of San Francisco?")

{'query': 'Who is the current mayor of San Francisco?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Mayor_of_San_Francisco',
   'title': 'Mayor of San Francisco - Wikipedia',
   'content': 'The current mayor is Democrat Daniel Lurie.',
   'score': 0.9994253,
   'raw_content': None},
  {'url': 'https://apnews.com/article/san-francisco-new-mayor-liberal-city-81ea0a7b37af6cbb68aea7ef5cc6a4f0',
   'title': "San Francisco's new mayor is starting to unite the fractured city",
   'content': 'San Francisco Mayor Daniel Lurie, a political newcomer and Levi Strauss heir, has marked his first 100 days with a hands-on, business-friendly approach.',
   'score': 0.9993538,
   'raw_content': None},
  {'url': 'https://www.sf.gov/departments--office-mayor',
   'title': 'Office of the Mayor - SF.gov',
   'content': 'Daniel Lurie is the 46th Mayor of the City and County of San Francisco.',
   'score': 0.99620515,
   'raw_content': None

In [9]:
agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search]
)

question = HumanMessage(content="Who is the current mayor of San Francisco?")

response = agent.invoke(
    {"messages": [question]}
)
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content='Who is the current mayor of San Francisco?', additional_kwargs={}, response_metadata={}, id='62ba3ac5-e30b-4086-9b36-060c8d65df54'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 475, 'prompt_tokens': 133, 'total_tokens': 608, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpiJMAM3WUe2TTAWkHPDeFg3xqq5K', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4818-109c-7e10-b98b-fa312582ab31-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'Current mayor of San Francisco'}, 'id': 'call_l6THHyHBoZT5ijKaU2uCrouM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, 'output_toke

You can view each run of the LangChain using traces here: https://smith.langchain.com/public/59432173-0dd6-49e8-9964-b16be6048426/r

Helps to debug and visualise the response['messages'] including response times and results.

Requirements:
- Langsmith API key

## How to give Agent short term memory for conversations

The state is the memory of the agent but the information is not being saved so each run wipes the memory of the agent. To solve this, we use `check_pointer` that allocates a `thread_id` to the conversation so it has state memory of what was mentioned previously.

In [11]:
from langgraph.checkpoint.memory import InMemorySaver  


agent = create_agent(
    "gpt-5-nano",
    checkpointer=InMemorySaver(),  
)

from langchain.messages import HumanMessage

question = HumanMessage(content="Hello my name is Seán and my favourite colour is green")
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [question]},
    config,  
)

pprint(response)

{'messages': [HumanMessage(content='Hello my name is Seán and my favourite colour is green', additional_kwargs={}, response_metadata={}, id='4e78b17f-5a27-428f-9d9b-f15220275f6c'),
              AIMessage(content='Nice to meet you, Seán! Green is a great color—calming and connected to nature. Do you have a favorite shade of green, or something you especially like about it? What would you like to chat about today—colors in general, Ireland, hobbies, or anything you need help with?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 648, 'prompt_tokens': 18, 'total_tokens': 666, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpiPKuFgjOCrYl4Nb1ON6NdLZQS5v', 'service_tier': 'defau

In [12]:
question = HumanMessage(content="What's my favourite colour?")

response = agent.invoke(
    {"messages": [question]},
    config,  
)

pprint(response)

{'messages': [HumanMessage(content='Hello my name is Seán and my favourite colour is green', additional_kwargs={}, response_metadata={}, id='4e78b17f-5a27-428f-9d9b-f15220275f6c'),
              AIMessage(content='Nice to meet you, Seán! Green is a great color—calming and connected to nature. Do you have a favorite shade of green, or something you especially like about it? What would you like to chat about today—colors in general, Ireland, hobbies, or anything you need help with?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 648, 'prompt_tokens': 18, 'total_tokens': 666, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpiPKuFgjOCrYl4Nb1ON6NdLZQS5v', 'service_tier': 'defau

Since `thread_id = 1`, our agent remembers the conversation due to the thread id.

# Multimodal messages

There can be multiple inputs for LLM like image and audio input. We encode the image and audio files in Base 64.

## Text input

In [17]:
from langchain.agents import create_agent

agent = create_agent(
    model='gpt-5-nano',
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
)

from langchain.messages import HumanMessage

question = HumanMessage(content=[
    {"type": "text", "text": "What is the capital of The Moon?"}
])

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

In this setting, the capital of The Moon is Lunaris Prime.

- Location: Nestled on the southern polar rim of Shackleton Crater, where near-constant sunlight cycles meet abundant water ice from the pole.
- Governance: The seat of the Lunar Covenant, a council-driven capital that coordinates mining, research, and orbital trade.
- Architecture: A crucible of glassy domes and latticework towers built into the crater rim, shielded by regolith walls; streets glow with pale mineral lamps.
- Economy/Technology: Ice harvesting and solar energy generation power a dense urban core; advanced 3D-printed infrastructure and AI-managed transit crisscross the city.
- Landmarks: The Beacon Spire (timekeeping and communications), the Archive Dome (the Moon’s historical records and scientific data), and the Lumen Market (a glass-walled bazaar of tech and culture).
- Vibe: A pristine, quiet metropolis that hums with technicians, scholars, and diplomats under a soft earthshine glow, where life rhythms follo

## Image input

In [13]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [14]:
print(uploader.value)

({'name': 'core_vs_page_dom_corr.png', 'type': 'image/png', 'size': 362267, 'content': <memory at 0x10d6aebc0>, 'last_modified': datetime.datetime(2025, 12, 16, 20, 38, 46, 728000, tzinfo=datetime.timezone.utc)},)


In [15]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [18]:
multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me about this capital"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

response = agent.invoke(
    {"messages": [multimodal_question]}
)

print(response['messages'][-1].content)

Here’s a portrait of the capital that sits behind that heatmap of metrics—Corelia, the shimmering heart of the Nexus Confederacy.

Name and role
- Corelia, Capital of the Nexus Confederacy. It’s not just a city, but a living network: a capital built to optimize every flow of data, people, and energy. It’s where policy is tested in real time by the city’s own neural grid, and where citizens learn to live with the pulse of the Net as a public utility.

How it looks and feels
- The Core at the heart: A towering spire called The Core dominates the skyline. It isn’t only a monument; it’s a distributed data center made visible—piercing glass, lattice of light threads, and a crown of biosynthetic vines that regulate microclimate and air quality. At dusk, the Core hums with a faint aurora of data streams.
- Districts named after the vitals: The city is organized around “districts” named after core web vitals and protocol concepts, a playful, practical nod to its raison d’être:
  - Duration Dis

## Audio Input

In [19]:
import sounddevice as sd
from scipy.io.wavfile import write
import base64
import io
import time
from tqdm import tqdm

# Recording settings
duration = 5  # seconds
sample_rate = 44100

print("Recording...")
audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
# Progress bar for the duration
for _ in tqdm(range(duration * 10)):   # update 10× per second
    time.sleep(0.1)
sd.wait()
print("Done.")

# Write WAV to an in-memory buffer
buf = io.BytesIO()
write(buf, sample_rate, audio)
wav_bytes = buf.getvalue()

aud_b64 = base64.b64encode(wav_bytes).decode("utf-8")

Recording...


100%|██████████| 50/50 [00:05<00:00,  9.58it/s]


Done.


In [20]:
agent = create_agent(
    model='gpt-4o-audio-preview',
)

multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me about this audio file"},
    {"type": "audio", "base64": aud_b64, "mime_type": "audio/wav"}
])

response = agent.invoke(
    {"messages": [multimodal_question]}
)

print(response['messages'][-1].content)

Whiskers twitch in morning light,  
Soft-pawed hunters, sleek and bright.  
Purring gently, tails held high,  
Silent shadows passing by.

Moonlit prowlers on the fence,  
Curled up napping, so intense.  
From playful pounce to graceful leap,  
In sunny spots, they fall asleep.

Eyes like jewels, green or gold,  
Mysteries refined, untold.  
Through every meow, a story weaves,  
Of cozy hearths and autumn leaves.

Companions quiet, yet so bold,  
In hearts and homes, their warmth they hold.  
Majestic, gentle, wise, and free—  
A cat’s soft purr is poetry.


# Agent Project

In [7]:
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pprint import pprint
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langgraph.checkpoint.memory import InMemorySaver  

load_dotenv()

True

In [8]:
# tavily search API is used to search the web in a LLM friendly way
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [ ]:
agent = create_agent(
    model="gpt-5-nano",
    tools = [web_search],
    checkpointer=InMemorySaver(),  
    streaming=True,
    max_token = 50,
    system_prompt = """
        You are a personal nutritionist who can give suggestions on food recipes that is leftover in the fridge or cupboards of the user.
        You need to ask the user what their nutrional goals are and what human digestion issues they may have in case they have food intollerance.
        They may have a disease like Crohns disease or Irritible Bowel Syndrome.
        """
)

question = HumanMessage(content="Hello my name is Sajid and I'm interested in some recipes for dinner today!")
config = {"configurable": {"thread_id": "123"}}

response = agent.invoke(
    {"messages": [question]},
    config,  
)


In [12]:
question = HumanMessage(content="Hey, so I want to gain weight, I have Crohn's, I have some chicken wings, some pasta and some milk. I have 15 minutes to cook and I need 2 servings. I love indian and italian cusine with halal options.")

response = agent.invoke(
    {"messages": [question]},
    config,  
)

pprint(response)

{'messages': [HumanMessage(content="Hello my name is Sajid and I'm interested in some recipes for dinner today!", additional_kwargs={}, response_metadata={}, id='c132bf92-94fb-4aa0-9110-e1216e4fd7fa'),
              AIMessage(content='Hi Sajid! I’d love to help you figure out dinner today. To tailor the ideas to you, could you share a few details?\n\n- What are your current nutritional goals? (e.g., maintain weight, lose weight, build muscle, balanced meals)\n- Do you have any digestion issues or intolerances? (e.g., IBS, Crohn’s, celiac, lactose intolerance, GERD)\n- What ingredients do you have on hand? List proteins, veggies, grains, and pantry items you’d like to use.\n- How much time do you have to cook? (e.g., 15 min, 30 min, or longer)\n- How many servings do you need?\n- Any cuisine preferences or spice tolerance?\n\nIf you’re not sure about ingredients yet, I can also suggest 3 flexible dinner ideas that usually work with common leftovers. For example:\n- Quick protein + veg s

In [13]:
print(response['messages'][-1].content)

Hi Sajid! Great—thanks for the details. Here are two quick, halal-friendly dinner ideas that fit your 15-minute window, use chicken wings, pasta, and milk, and are tuned for weight gain with Crohn’s in mind. Both are designed for 2 servings and draw on Indian or Italian flavors you mentioned.

Option 1: Indian-inspired Creamy Chicken Wings with Pasta (mild, dairy-inclusive)
Ingredients (2 servings)
- 8 halal chicken wings (drumettes)
- 200 g pasta (your choice)
- 1 cup milk (240 ml; use whole or lactose-free if preferred)
- 2–3 cloves garlic, minced
- 1/2 onion, finely chopped (optional)
- 1–2 tsp garam masala
- 1/2 tsp ground cumin
- 1/4 tsp turmeric
- 1/2 tsp paprika or chili powder (optional or use less for milder spice)
- 1–2 tbsp oil
- 1–2 tbsp tomato paste or a splash of crushed tomatoes (optional)
- Salt and pepper to taste
- Fresh cilantro or parsley for garnish

Steps
1) Bring a pot of salted water to boil for the pasta and cook to al dente according to package directions. Res

# Agent Graph

You can also look at Langchain UI to view agents with their graphs and a web interface.

Requirements:
1. Create a `.py` file that contains your agent
2. Copy the `langgraph.json` file with your agent and env pointed.
3. Open your terminal, working directory, `langgraph dev`

# Module 2: Advanced Agent

## Model Context Protocol (MCP)

### What is MCP?
An open protocol that standardizes how the LLM applications connect to and work with your tools and data sources.
Anology: USB cable! Allows you to connect microphone to computer with one cable, a standard wire. MCP is a universal model protocol, MCP server allows you to insert others servers to yours!

You can find open source MCP servers here: mcp.so/servers


## How to create a MCP client

In [ ]:
# example 1 of a MCP client
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio", # can be standard IO or HTTP depending on the server
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [ ]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt=prompt
)

from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

from pprint import pprint

pprint(response)

MCP Client = The agent
MCP Server = The conversation rulebook to connecting to the server, how to message back and forth and how tools, data and actions are shared.

In [ ]:
# example - Notion MCP server
{
  "mcpServers": {
    "notionMCP": {
      "command": "npx",
      "args": ["-y", "mcp-remote", "https://mcp.notion.com/mcp"]
    }
  }
}

In [ ]:
# example - Travel Agent
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

## How to set up a context schema

Allows the human to fill in a context schema for the agent to read and inform it's action and responses. We use the function `ToolRuntime` to allow the agent to access only the context of information it needs so we don't overload the agent.

In [15]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

In [16]:
from langchain.tools import tool, ToolRuntime

# storing the colours from tool decorator
@tool
def get_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the favourite colour of the user"""
    return runtime.context.favourite_colour

@tool
def get_least_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the least favourite colour of the user"""
    return runtime.context.least_favourite_colour

In [17]:
agent = create_agent(
    model="gpt-5-nano",
    tools=[get_favourite_colour, get_least_favourite_colour],
    context_schema=ColourContext
)

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

pprint(response)

/Users/sajidahmed/Desktop/VS Code/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=ColourContext(favourite_c...vourite_colour='yellow'), input_type=ColourContext])
  return self.__pydantic_serializer__.to_python(


{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='2c6fd79f-9b5d-452e-b350-a45a713ed202'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 149, 'prompt_tokens': 149, 'total_tokens': 298, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cq1vgXxbbnvsEmp80Q2xm8Y0HE9Nx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4c96-c321-7380-86c5-97c1e7d40f34-0', tool_calls=[{'name': 'get_favourite_colour', 'args': {}, 'id': 'call_0dux9GAMD63BZqm2C9WfPd2O', 'type': 'tool_call'}], usage_metadata={'input_tokens': 149, 'output_tokens': 149, 'total_to

In [ ]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    # context can be overwritten
    context=ColourContext(favourite_colour="green")
)

pprint(response)

/Users/sajidahmed/Desktop/VS Code/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=ColourContext(favourite_c...vourite_colour='yellow'), input_type=ColourContext])
  return self.__pydantic_serializer__.to_python(


{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='18b9128e-8fa2-4885-8bcf-83980263abdb'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 149, 'prompt_tokens': 149, 'total_tokens': 298, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cq1vpHq7lyc2zuwRQSqy524a3vQ1a', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4c96-dfbf-7bb1-8650-577aa20b6fa4-0', tool_calls=[{'name': 'get_favourite_colour', 'args': {}, 'id': 'call_p3xUsDVE0lQIsWncbrU1Ip1Y', 'type': 'tool_call'}], usage_metadata={'input_tokens': 149, 'output_tokens': 149, 'total_to

Context schemas are immutable, the agent cannot override my favourite colour naturally. But using the agent's state, we can have the agent keep track of our historical information like my favourite colour.

In [ ]:
from langchain.agents import AgentState

# 1. create a state called custom state for the field you want to track
class CustomState(AgentState):
    favourite_colour: str

In [ ]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

# 2. create a tool decorator for the agent to keep track of the custom field
@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    # Command is used to allow the agent to update the favourite_colour variable
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [21]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    "gpt-5-nano",
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [22]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='179c3192-e09e-4c59-a946-3bd22accba8b'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 347, 'prompt_tokens': 141, 'total_tokens': 488, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cq1ykeIQYOQXMghi0WzzLnEbmYTe1', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4c99-abfd-7ed1-913f-a62b74c918ae-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': 'call_8YHabbjGaYEPloCLluyeXOwJ', 'type': 'tool_call'}], usage_me

In [23]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='e337f614-4a97-413f-a3b4-8a261258af16'),
              AIMessage(content="I'm doing well, thanks for asking! How about you?  \nWhat can I help you with today—any questions, ideas you want to brainstorm, or a task you’d like help with? Some examples: explain concepts, draft emails, brainstorm names or ideas, code help, or translations.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 260, 'prompt_tokens': 142, 'total_tokens': 402, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cq1z7JTaeZmLPKmZzwV1ljdf5V3W7', 'service_tier': 'default', 'finish

In [ ]:
# 3. Allows agent to read the state of the field
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    "gpt-5-nano",
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [ ]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

In [ ]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

## Creating SubAgents

The performance of an agent suffers if we have a long workflow, it is better to modularise the agents to be great at one specific task. One agent for editing, one agent for research etc

### Supervisor - Subagent model

One supervisor agent who orchestrates the tasks to the subagent using Langchain. 

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain.tools import tool

# one tool for square root function
@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

# one tool for square function
@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [ ]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model='gpt-5-nano',
    tools=[square_root]
)

subagent_2 = create_agent(
    model='gpt-5-nano',
    tools=[square]
)

In [ ]:
from langchain.messages import HumanMessage

# subagent 1
@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

# subagent 2
@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent (supervisor)

main_agent = create_agent(
    model='gpt-5-nano',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

Tracing is used here in langSmith to see and debug each output

In [24]:
# Wedding planner task
# agent 1 = flights
# agent 2 = venue
# agent 3 = playlist

from dotenv import load_dotenv
load_dotenv()

True

In [25]:
# agent 1 = flights

from langchain_mcp_adapters.client import MultiServerMCPClient

travel_client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)


# get tools
tools = await travel_client.get_tools()

# get resources
resources = await travel_client.get_resources("travel_server")

# get prompts
prompt = await travel_client.get_prompt("travel_server", "prompt")
prompt = prompt[0].content

  + Exception Group Traceback (most recent call last):
  |   File "/Users/sajidahmed/Desktop/VS Code/lca-lc-foundations/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/4l/356ftc5d0wsd3c6hcfb63f4m0000gn/T/ipykernel_87523/33727243.py", line 19, in <module>
  |     resources = await travel_client.get_resources("travel_server")
  |                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/sajidahmed/Desktop/VS Code/lca-lc-foundations/.venv/lib/python3.12/site-packages/langchain_mcp_adapters/client.py", line 238, in get_resources
  |     async with self.session(server_name) as session:
  |   File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(value)
  |   File "/Users/sajidahmed/Desktop/VS Code/lca-lc-foundations/.venv/lib/python3.12/site-packa

In [ ]:
# agent 2 = venues from the web
from typing import Dict, Any
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [ ]:
# agent 3 = playlist mustic
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

In [ ]:
from langchain.agents import AgentState

# creating a wedding state to track
class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

In [ ]:
from langchain.agents import create_agent

# create subagents

travel_agent = create_agent(
    model='gpt-5-nano',
    tools=[tools],
    system_prompt = """
        You are a travel agent. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
    """ 
)

venue_agent = create_agent(
    model='gpt-5-nano',
    tools=[web_search],
    system_prompt = """
       You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
        """
)

playlist_agent = create_agent(
    model='gpt-5-nano',
    tools=[query_playlist_db],
    system_prompt="""
    You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options.
    """
)

In [ ]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

# defining the tools for the supervisor agent to call and run
@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]})
    return response['messages'][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "genre": genre, 
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )


In [ ]:
from langchain.agents import create_agent

# supervisor orchestration
coordinator = create_agent(
    model="gpt-5-nano",
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt="""
    You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me.
    """
)

In [ ]:
from langchain.messages import HumanMessage

# testing 
response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre")],
    }
)

In [ ]:
from pprint import pprint

pprint(response)

In [ ]:
print(response["messages"][-1].content)